In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
#Data import and feature additions
beer_df = pd.read_csv('https://query.data.world/s/nuub3qupegsd33g3nimifjpajqeq2o')
beer_df.dropna(inplace=True)
beer_df.drop_duplicates(inplace=True)
beer_df["beer_strength"] = pd.cut(beer_df["beer_abv"], [0,1.9,3.9,6,9,58], 
                                labels=["Light", "Mid", "Full", "Heavy", "Very Strong"])
mod_beer_df =beer_df
conditions =[
    (mod_beer_df.beer_style.str.contains('ale', case=False)),
    (mod_beer_df.beer_style.str.contains('dark ale', case=False)),
    (mod_beer_df.beer_style.str.contains('ipa', case=False)),
    (mod_beer_df.beer_style.str.contains("pilsener", case=False)),
    (mod_beer_df.beer_style.str.contains("pilsner", case=False)),
    (mod_beer_df.beer_style.str.contains("barleywine", case=False)),
    (mod_beer_df.beer_style.str.contains("bitter", case=False)),
    (mod_beer_df.beer_style.str.contains("porter", case=False)),
    (mod_beer_df.beer_style.str.contains("kölsch", case=False)),
    (mod_beer_df.beer_style.str.contains('pel', case=False)),
    (mod_beer_df.beer_style.str.contains('lager', case=False)),
    (mod_beer_df.beer_style.str.contains('stout', case=False)),
]

styles = ['ale','dark ale','ipa',"pilsener","pilsener","barleywine","bitter","porter","kölsch","trappist",'lager','stout']
mod_beer_df["meta_style"] = np.select(conditions, styles, default='exotic')
beer_sub_grp = mod_beer_df[["beer_name", "review_overall", "review_aroma", "review_appearance", "review_palate", "review_taste"]]
grp_beers_mean= pd.DataFrame(beer_sub_grp.groupby(["beer_name"]).mean())
mod_beer_df = pd.merge(mod_beer_df, grp_beers_mean, on='beer_name',how='outer',suffixes=('_OG','_MEAN'))
val_info = pd.DataFrame(mod_beer_df[["review_overall_OG", "review_aroma_OG", "review_appearance_OG", "review_palate_OG", "review_taste_OG"]])
mod_beer_df["sub_review_score"] = val_info.mean(axis=1)